### ***Installation et importation***

In [30]:

import pandas as pd
import numpy as np
import duckdb
import json
import re
from datetime import datetime
import os
from pathlib import Path

BASE_DIR = Path.cwd().parent  # Remonte d'un niveau si dans notebooks/
DATA_DIR = BASE_DIR / "data"
DB_DIR = DATA_DIR / "database"

# Création des répertoires si nécessaire
DB_DIR.mkdir(parents=True, exist_ok=True)

# Chemins des fichiers
CSV_PATH = DATA_DIR / "dataset_final.csv"
DB_PATH = DB_DIR / "election_ci.db"
SCHEMA_PATH = Path("E:/Projet_artefact/src/schema_for_agent.json")


print(" Structure des répertoires :")
print(f"   Base : {BASE_DIR}")
print(f"   Données : {DATA_DIR}")
print(f"   CSV source : {CSV_PATH}")
print(f"   Base de données : {DB_PATH}")
print(f"   Schéma JSON : {SCHEMA_PATH}")

# Vérification des fichiers d'entrée
if not CSV_PATH.exists():
    raise FileNotFoundError(f" Fichier CSV non trouvé : {CSV_PATH}")

df_clean = pd.read_csv(CSV_PATH)

 Structure des répertoires :
   Base : e:\Projet_artefact\src
   Données : e:\Projet_artefact\src\data
   CSV source : e:\Projet_artefact\src\data\dataset_final.csv
   Base de données : e:\Projet_artefact\src\data\database\election_ci.db
   Schéma JSON : E:\Projet_artefact\src\schema_for_agent.json


##### ***Connexion à DuckDB***

In [ ]:
# ## 3.1 Connexion à DuckDB
print(f"\n Création de la base de données DuckDB...")
print(f"   Chemin : {DB_PATH}")

# Supprimer la base existante si nécessaire
if DB_PATH.exists():
    print(f"     Base existante détectée, remplacement...")
    DB_PATH.unlink()

# Connexion
conn = duckdb.connect(str(DB_PATH))
print(" Connexion établie à DuckDB")

##### ***Création de la table principale***

In [ ]:
# Créer la table à partir du DataFrame nettoyé
conn.execute("""
CREATE OR REPLACE TABLE raw_election_results AS 
SELECT * FROM df_clean
""")

# Vérification
table_info = conn.execute("SELECT COUNT(*) as total_rows FROM raw_election_results").fetchone()
print(f" Table 'raw_election_results' créée avec {table_info[0]:,} lignes")

# Afficher le schéma
print("\n Schéma de la table :")
schema = conn.execute("DESCRIBE raw_election_results").fetchdf()
print(schema.to_string())

 Table 'raw_election_results' créée avec 1,125 lignes

 Schéma de la table :
                   column_name column_type null   key default extra
0                       region     VARCHAR  YES  None    None  None
1              circonscription     VARCHAR  YES  None    None  None
2          nombre_bureaux_vote      BIGINT  YES  None    None  None
3                     inscrits      BIGINT  YES  None    None  None
4                      votants      BIGINT  YES  None    None  None
5        taux_de_participation      DOUBLE  YES  None    None  None
6               bulletins_nuls      BIGINT  YES  None    None  None
7           suffrages_exprimes      BIGINT  YES  None    None  None
8             bulletins_blancs      BIGINT  YES  None    None  None
9           pourcentage_blancs      DOUBLE  YES  None    None  None
10                       parti     VARCHAR  YES  None    None  None
11                    candidat     VARCHAR  YES  None    None  None
12                        voix      BIG

In [ ]:
print("1. Vue principale vw_results_clean (CORRIGÉE)...")
#conn.execute("DROP VIEW IF EXISTS vw_results_clean")
#conn.execute("DROP VIEW IF EXISTS vw_results_clean")
conn.execute("""
CREATE VIEW vw_results_clean AS
WITH filled_data AS (
    SELECT *,
        -- Remplir seulement circonscription_id vide avec la précédente valeur
        CASE 
            WHEN circonscription_id IS NOT NULL AND circonscription_id != '' 
            THEN circonscription_id
            ELSE LAST_VALUE(circonscription_id IGNORE NULLS) OVER (
                PARTITION BY region 
                ORDER BY row_id 
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            )
        END as circonscription_id_filled
    FROM raw_election_results
    WHERE candidat IS NOT NULL AND candidat != ''
),
ranked_data AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY circonscription_id_filled 
            ORDER BY CAST(voix AS BIGINT) DESC
        ) as rang
    FROM filled_data
    WHERE circonscription_id_filled IS NOT NULL 
    AND circonscription_id_filled != ''
)
SELECT 
    row_id,
    region,
    region_normalized,
    region_slug,
    circonscription,
    circonscription_normalized,
    circonscription_id_filled as circonscription_id,
    CAST(nombre_bureaux_vote AS DOUBLE) as nombre_bureaux_vote,
    CAST(inscrits AS DOUBLE) as inscrits,
    CAST(votants AS DOUBLE) as votants,
    CAST(taux_de_participation AS DOUBLE) as taux_de_participation,
    CAST(bulletins_nuls AS DOUBLE) as bulletins_nuls,
    CAST(suffrages_exprimes AS DOUBLE) as suffrages_exprimes,
    bulletins_blancs,
    CAST(pourcentage_blancs AS DOUBLE) as pourcentage_blancs,
    parti,
    parti_normalized,
    parti_standardized,
    parti_slug,
    candidat,
    candidat_normalized,
    CAST(voix AS BIGINT) as voix,
    CAST(pourcentage_voix AS DOUBLE) as pourcentage_voix,
    elu,
    -- Seul le premier de chaque circonscription est élu
    CAST(CASE WHEN rang = 1 THEN 1 ELSE 0 END AS BIGINT) as est_elu,
    processed_at
FROM ranked_data
""")
print("   Vue créée (circonscription_id remplis automatiquement)")

1. Vue principale vw_results_clean (CORRIGÉE)...
   Vue créée (circonscription_id remplis automatiquement)


### ***Creation des vues***

##### ***Vue principale vw_results_clean***

In [ ]:
print("1. Vue principale vw_results_clean...")

conn.execute("DROP VIEW IF EXISTS vw_results_clean")

conn.execute("""
CREATE VIEW vw_results_clean AS
WITH filled AS (
    SELECT *,
        COALESCE(circonscription, 
                LAST_VALUE(circonscription IGNORE NULLS) OVER w) as circonscription_filled,
        COALESCE(circonscription_normalized,
                LAST_VALUE(circonscription_normalized IGNORE NULLS) OVER w) as circonscription_normalized_filled
    FROM raw_election_results
    WHERE candidat IS NOT NULL AND candidat != ''
    WINDOW w AS (PARTITION BY region ORDER BY row_id)
),
with_id AS (
    SELECT *,
        LOWER(CONCAT(region_normalized, '_',
            REGEXP_REPLACE(REGEXP_REPLACE(circonscription_normalized_filled, '[^a-zA-Z0-9]+', '_'), '_+', '_')
        )) as circonscription_id
    FROM filled
    WHERE circonscription_filled IS NOT NULL
),
ranked AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY circonscription_id ORDER BY CAST(voix AS BIGINT) DESC) as rang
    FROM with_id
)
SELECT 
    row_id, region, region_normalized, region_slug,
    circonscription_filled as circonscription,
    circonscription_normalized_filled as circonscription_normalized,
    circonscription_id,
    CAST(nombre_bureaux_vote AS DOUBLE) as nombre_bureaux_vote,
    CAST(inscrits AS DOUBLE) as inscrits,
    CAST(votants AS DOUBLE) as votants,
    CAST(taux_de_participation AS DOUBLE) as taux_de_participation,
    CAST(bulletins_nuls AS DOUBLE) as bulletins_nuls,
    CAST(suffrages_exprimes AS DOUBLE) as suffrages_exprimes,
    bulletins_blancs,
    CAST(pourcentage_blancs AS DOUBLE) as pourcentage_blancs,
    parti, parti_normalized, parti_standardized, parti_slug,
    candidat, candidat_normalized,
    CAST(voix AS BIGINT) as voix,
    CAST(pourcentage_voix AS DOUBLE) as pourcentage_voix,
    elu,
    CAST(CASE WHEN rang = 1 THEN 1 ELSE 0 END AS BIGINT) as est_elu,
    processed_at
FROM ranked
""")

print(f"   ✓ Vue créée (types de données alignés avec le schéma)")


1. Vue principale vw_results_clean...
   ✓ Vue créée (types de données alignés avec le schéma)


##### ***Vue des candidats élus vw_winners***

In [ ]:
print("\n2. Vue vw_winners...")
conn.execute("DROP VIEW IF EXISTS vw_winners")
conn.execute("""
CREATE VIEW vw_winners AS
SELECT 
    row_id,
    region,
    region_normalized,
    circonscription,
    circonscription_normalized,
    circonscription_id,
    parti,
    parti_standardized,
    candidat,
    candidat_normalized,
    CAST(voix AS BIGINT) as voix,
    CAST(pourcentage_voix AS DOUBLE) as pourcentage_voix,
    ROW_NUMBER() OVER (PARTITION BY circonscription_id ORDER BY voix DESC) as rang_circonscription
FROM vw_results_clean
WHERE est_elu = 1
""")
print("    Vue créée")



2. Vue vw_winners...
    Vue créée


##### ***Vue des statistiques de participation vw_turnout***

In [ ]:

print("\n3. Vue vw_turnout...")
conn.execute("DROP VIEW IF EXISTS vw_turnout")
conn.execute("""
CREATE VIEW vw_turnout AS
SELECT 
    region,
    circonscription,
    circonscription_id,
    nombre_bureaux_vote,
    inscrits,
    votants,
    taux_de_participation,
    bulletins_nuls,
    suffrages_exprimes,
    -- Convertir bulletins_blancs en DOUBLE
    CAST(bulletins_blancs AS DOUBLE) as bulletins_blancs,
    pourcentage_blancs
FROM (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY circonscription_id ORDER BY row_id) as rn
    FROM vw_results_clean
    WHERE circonscription_id IS NOT NULL
    AND nombre_bureaux_vote IS NOT NULL  -- Prendre seulement les lignes avec stats
) t
WHERE rn = 1
""")
print("   Vue créée")



3. Vue vw_turnout...
   Vue créée


##### ***Vue des statistiques par parti vw_party_stats***

In [ ]:

print("\n4. Vue vw_party_stats...")
conn.execute("DROP VIEW IF EXISTS vw_party_stats")
conn.execute("""
CREATE VIEW vw_party_stats AS
WITH party_agg AS (
    SELECT 
        parti_standardized as parti,
        circonscription_id,
        COUNT(*) as candidats_ici,
        SUM(voix) as voix_ici,
        MAX(est_elu) as a_gagne_ici
    FROM vw_results_clean
    WHERE parti_standardized IS NOT NULL
    GROUP BY parti_standardized, circonscription_id
)
SELECT 
    parti,
    COUNT(DISTINCT circonscription_id) as circonscriptions_presentes,
    CAST(SUM(a_gagne_ici) AS HUGEINT) as sieges_remportes,
    CAST(
        CASE WHEN COUNT(DISTINCT circonscription_id) > 0 
             THEN CAST(SUM(a_gagne_ici) AS FLOAT) / COUNT(DISTINCT circonscription_id) * 100
             ELSE 0 
        END AS FLOAT
    ) as taux_victoire_pourcent,
    CAST(SUM(voix_ici) AS HUGEINT) as total_voix,
    CAST(SUM(candidats_ici) AS HUGEINT) as total_candidats,
    CAST(AVG(voix_ici) AS DOUBLE) as voix_moyennes_par_circo
FROM party_agg
GROUP BY parti
HAVING COUNT(DISTINCT circonscription_id) > 0
ORDER BY sieges_remportes DESC
""")
print("    Vue créée")



4. Vue vw_party_stats...
    Vue créée


##### ***Vue des statistiques par région vw_region_stats***

In [ ]:

print("\n5. Vue vw_region_stats...")
conn.execute("DROP VIEW IF EXISTS vw_region_stats")
conn.execute("""
CREATE VIEW vw_region_stats AS
WITH region_turnout AS (
    -- Calculer correctement les totaux avec bulletins_blancs en DOUBLE
    SELECT 
        region,
        COUNT(DISTINCT circonscription_id) as nombre_circonscriptions,
        SUM(nombre_bureaux_vote) as total_bureaux_vote,
        SUM(inscrits) as total_inscrits,
        SUM(votants) as total_votants,
        AVG(taux_de_participation) as taux_participation_moyen,
        SUM(bulletins_nuls) as total_bulletins_nuls,
        SUM(CAST(bulletins_blancs AS DOUBLE)) as total_bulletins_blancs,
        SUM(suffrages_exprimes) as total_suffrages_exprimes
    FROM vw_results_clean
    WHERE nombre_bureaux_vote IS NOT NULL  -- Seulement les lignes avec stats
    GROUP BY region
),
region_election AS (
    SELECT 
        region,
        COUNT(DISTINCT circonscription_id) as circonscriptions_avec_resultats,
        COUNT(*) as total_candidats,
        SUM(est_elu) as total_elus,
        SUM(voix) as total_voix_exprimees
    FROM vw_results_clean
    GROUP BY region
)
SELECT 
    COALESCE(rt.region, re.region) as region,
    CAST(rt.nombre_circonscriptions AS BIGINT) as nombre_circonscriptions,
    CAST(rt.total_bureaux_vote AS DOUBLE) as total_bureaux_vote,
    CAST(rt.total_inscrits AS DOUBLE) as total_inscrits,
    CAST(rt.total_votants AS DOUBLE) as total_votants,
    CAST(rt.taux_participation_moyen AS DOUBLE) as taux_participation_moyen,
    CAST(rt.total_bulletins_nuls AS DOUBLE) as total_bulletins_nuls,
    CAST(rt.total_bulletins_blancs AS DOUBLE) as total_bulletins_blancs,
    -- CORRECTION : Suffrages exprimés = total_voix_exprimees
    CAST(re.circonscriptions_avec_resultats AS BIGINT) as circonscriptions_avec_resultats,
    CAST(re.total_candidats AS BIGINT) as total_candidats,
    CAST(re.total_elus AS HUGEINT) as total_elus,
    CAST(re.total_voix_exprimees AS HUGEINT) as total_voix_exprimees
FROM region_turnout rt
FULL OUTER JOIN region_election re ON rt.region = re.region
ORDER BY COALESCE(rt.region, re.region)
""")

print("    Vue créée")



5. Vue vw_region_stats...
    Vue créée


##### ***Vue de performance des candidats vw_candidate_performance***

In [ ]:

print("\n6. Vue vw_candidate_performance...")
conn.execute("DROP VIEW IF EXISTS vw_candidate_performance")
conn.execute("""
CREATE VIEW vw_candidate_performance AS
WITH ranked AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY circonscription_id ORDER BY voix DESC) as rang_circonscription
    FROM vw_results_clean
    WHERE voix > 0
)
SELECT 
    row_id,
    region,
    circonscription,
    circonscription_id,
    candidat,
    candidat_normalized,
    parti_standardized as parti,
    CAST(voix AS BIGINT) as voix,
    CAST(pourcentage_voix AS DOUBLE) as pourcentage_voix,
    CAST(est_elu AS BIGINT) as est_elu,
    CAST(rang_circonscription AS BIGINT) as rang_circonscription,
    CASE 
        WHEN rang_circonscription = 1 THEN 'Gagnant'
        WHEN rang_circonscription <= 3 THEN 'Top 3'
        ELSE 'Autre'
    END as performance_categorie,
    CASE 
        WHEN est_elu = 1 AND rang_circonscription = 1 THEN 'Élu (1er)'
        WHEN est_elu = 1 THEN 'Élu (autre rang)'
        WHEN rang_circonscription = 1 THEN '1er non-élu'
        ELSE 'Candidat'
    END as statut_detail
FROM ranked
""")
print("   Vue créée")



6. Vue vw_candidate_performance...
   Vue créée


##### ***Vue pour recherche RAG vw_rag_search***

In [ ]:

print("\n7. Vue vw_rag_search...")
conn.execute("DROP VIEW IF EXISTS vw_rag_search")
conn.execute("""
CREATE VIEW vw_rag_search AS
SELECT 
    row_id,
    region,
    circonscription,
    parti_standardized as parti,
    candidat,
    CAST(voix AS BIGINT) as voix,
    CAST(pourcentage_voix AS DOUBLE) as pourcentage_voix,
    CAST(est_elu AS BIGINT) as est_elu,
    elu,
    CONCAT(
        'Région: ', COALESCE(region, ''), ' | ',
        'Circonscription: ', COALESCE(circonscription, ''), ' | ',
        'Parti: ', COALESCE(parti_standardized, ''), ' | ',
        'Candidat: ', COALESCE(candidat, ''), ' | ',
        'Voix: ', CAST(voix AS VARCHAR), ' (',
        CAST(ROUND(pourcentage_voix, 2) AS VARCHAR), '%) | ',
        'Statut: ', CASE WHEN est_elu = 1 THEN 'ÉLU' ELSE 'Non élu' END
    ) as search_text
FROM vw_results_clean
WHERE region IS NOT NULL AND candidat IS NOT NULL
""")
print("   Vue créée")



7. Vue vw_rag_search...
   Vue créée


##### ***Table de référence des partis dim_party***

In [ ]:

print("\n8. Table dim_party...")
conn.execute("DROP TABLE IF EXISTS dim_party")
conn.execute("""
CREATE TABLE dim_party AS
SELECT 
    CAST(ROW_NUMBER() OVER (ORDER BY parti_standardized) AS INTEGER) as party_id,
    parti_standardized as party_name,
    CAST(COUNT(*) AS BIGINT) as total_candidates,
    CAST(SUM(est_elu) AS BIGINT) as total_seats_won,
    CAST(SUM(voix) AS BIGINT) as total_votes,
    CAST(ROUND(AVG(pourcentage_voix), 2) AS DOUBLE) as avg_vote_percentage
FROM vw_results_clean
WHERE parti_standardized IS NOT NULL
GROUP BY parti_standardized
""")
print("   Table créée")



8. Table dim_party...
   Table créée


##### ***vue pour la recherche de partis***

In [ ]:

print("\n9. Vue vw_party_search...")
conn.execute("DROP VIEW IF EXISTS vw_party_search")
conn.execute("""
CREATE VIEW vw_party_search AS
SELECT 
    party_id,
    party_name as parti,
    total_candidates,
    total_seats_won as seats_won,
    total_votes,
    avg_vote_percentage,
    CASE 
        WHEN party_name = 'RHDP' THEN 'R.H.D.P,Rassemblement des Houphouëtistes,RASSEMBLEMENT DES HOUPHOUËTISTES POUR LA DÉMOCRATIE ET LA PAIX'
        WHEN party_name = 'INDEPENDANT' THEN 'INDÉPENDANT,IND,SANS ETIQUETTE,CANDIDAT INDÉPENDANT'
        WHEN party_name = 'PDCI-RDA' THEN 'PDCI,RDA,PARTI DÉMOCRATIQUE DE CÔTE D''IVOIRE'
        WHEN party_name = 'FPI' THEN 'FRONT POPULAIRE IVOIRIEN'
        WHEN party_name = 'ADCI' THEN 'ALLIANCE DES DÉMOCRATES DE CÔTE D''IVOIRE,ADCI - GP-PAIX - VALEUR'
        WHEN party_name = 'MGC' THEN 'MOUVEMENT DES GENS DE CÔTE D''IVOIRE'
        WHEN party_name = 'PRO CI' THEN 'PRO COTE DIVOIRE'
        WHEN party_name = 'PDCI - FPI - ADCI' THEN 'PDCI-FPI-ADCI'
        WHEN party_name = 'PDCI-RDA - EDS' THEN 'PDCI-RDA-EDS'
        ELSE party_name
    END as known_aliases,
    CONCAT(
        'Parti: ', party_name, ' | ',
        'Candidats: ', CAST(total_candidates AS VARCHAR), ' | ',
        'Sièges: ', CAST(total_seats_won AS VARCHAR), ' | ',
        'Voix: ', CAST(total_votes AS VARCHAR), ' | ',
        'Moyenne: ', CAST(ROUND(avg_vote_percentage, 2) AS VARCHAR), '%'
    ) as search_text
FROM dim_party
ORDER BY total_votes DESC
""")
print("   Vue créée")



9. Vue vw_party_search...
   Vue créée


### ***Liste complète des vues***

In [ ]:
print("\n Liste des vues créées :")
views = conn.execute("""
SELECT table_name as vue, 
       (SELECT COUNT(*) FROM (SELECT * FROM information_schema.tables t2 WHERE t2.table_name = t.table_name LIMIT 1)) as count
FROM information_schema.tables t
WHERE table_schema = 'main' 
AND table_type = 'VIEW'
ORDER BY table_name
""").fetchdf()

print(views.to_string(index=False))


 Liste des vues créées :
                     vue  count
vw_candidate_performance      1
         vw_party_search      1
          vw_party_stats      1
           vw_rag_search      1
         vw_region_stats      1
        vw_results_clean      1
              vw_turnout      1
              vw_winners      1


### ***TEST DE LA CONFORMITER et des QUESTIONS DU CAHIER DE CHARGES***

In [ ]:
print(" QUESTIONS DU CAHIER DES CHARGES - RÉPONSES COMPLÈTES")

questions = {
    "Q1": "Combien de sièges le RHDP a-t-il remportés ?",
    "Q2": "Top 10 des candidats par score dans la région AGNEBY-TIASSA",
    "Q3": "Taux de participation par région",
    "Q4": "Histogramme des gagnants par parti",
    "Q5": "Statistiques électorales globales"
}

# Q1 - Version CORRECTE
print(f"\n1. {questions['Q1']}")
q1_correct = conn.execute("""
SELECT 
    parti,
    sieges_remportes,
    CONCAT(
        CAST(sieges_remportes AS VARCHAR), 
        ' sièges sur 205 (',
        CAST(ROUND(sieges_remportes / 205.0 * 100, 1) AS VARCHAR),
        '%)'
    ) as resultat_formate,
    taux_victoire_pourcent,
    'Le RHDP a remporté ' || CAST(sieges_remportes AS VARCHAR) || 
    ' sièges sur 205 circonscriptions nationales.' as interpretation
FROM vw_party_stats
WHERE parti = 'RHDP'
""").fetchdf()
print(q1_correct.to_string(index=False))

# Q2 - Top 10 candidats AGNEBY-TIASSA
print(f"\n2. {questions['Q2']}")
q2 = conn.execute("""
SELECT 
    ROW_NUMBER() OVER (ORDER BY voix DESC) as rang,
    candidat, 
    parti, 
    voix, 
    pourcentage_voix,
    CASE WHEN est_elu = 1 THEN 'ÉLU' ELSE 'Non élu' END as statut
FROM vw_results_clean
WHERE region = 'AGNEBY-TIASSA'
ORDER BY voix DESC
LIMIT 10
""").fetchdf()
print(q2.to_string(index=False))

# Q3 - Taux de participation par région
print(f"\n3. {questions['Q3']} (Top 5)")
q3 = conn.execute("""
SELECT 
    ROW_NUMBER() OVER (ORDER BY taux_participation_moyen DESC) as rang,
    region, 
    ROUND(taux_participation_moyen, 2) as participation_moyenne,
    total_votants,
    total_inscrits,
    ROUND(total_votants / total_inscrits * 100, 2) as taux_reel
FROM vw_region_stats
ORDER BY taux_participation_moyen DESC
LIMIT 5
""").fetchdf()
print(q3.to_string(index=False))

# Q4 - Histogramme des gagnants par parti
print(f"\n4. {questions['Q4']}")
q4_correct = conn.execute("""
SELECT 
    ROW_NUMBER() OVER (ORDER BY sieges_remportes DESC) as rang,
    parti,
    sieges_remportes,
    ROUND(sieges_remportes / 205.0 * 100, 1) as pourcentage_sieges,
    total_voix,
    ROUND(total_voix / (SELECT SUM(total_voix) FROM vw_party_stats) * 100, 1) as pourcentage_voix,
    CASE 
        WHEN parti = 'RHDP' THEN 'Majoritaire'
        WHEN sieges_remportes >= 10 THEN 'Opposition significative'
        ELSE 'Minoritaire'
    END as categorie
FROM vw_party_stats
WHERE sieges_remportes > 0
ORDER BY sieges_remportes DESC
""").fetchdf()
print(q4_correct.to_string(index=False))

# Q5 - NOUVELLES STATISTIQUES GLOBALES
print(f"\n5. {questions['Q5']} - DONNÉES COMPLÈTES")
q5_stats = conn.execute("""
WITH stats_globales AS (
    SELECT 
        -- Nombre total de bureaux de vote
        CAST(SUM(total_bureaux_vote) AS BIGINT) as total_bureaux_vote,
        -- Nombre total d'inscrits
        CAST(SUM(total_inscrits) AS BIGINT) as total_inscrits,
        -- Nombre total de votants
        CAST(SUM(total_votants) AS BIGINT) as total_votants,
        -- Participation moyenne
        ROUND(AVG(taux_participation_moyen), 2) as taux_participation_moyen,
        -- Bulletins nuls totaux
        CAST(SUM(total_bulletins_nuls) AS BIGINT) as total_bulletins_nuls,
        -- Suffrages exprimés totaux
        (SELECT CAST(SUM(total_voix_exprimees) AS BIGINT) FROM vw_region_stats) as total_suffrages_exprimes,
        -- Bulletins blancs totaux
        CAST(SUM(total_bulletins_blancs) AS BIGINT) as total_bulletins_blancs,
        -- Calcul des pourcentages
        ROUND(SUM(total_votants) / SUM(total_inscrits) * 100, 2) as taux_participation_reel,
        ROUND(SUM(total_bulletins_nuls) / SUM(total_votants) * 100, 2) as pourcentage_nuls,
        ROUND(SUM(total_bulletins_blancs) / SUM(total_votants) * 100, 2) as pourcentage_blancs
    FROM vw_region_stats
),
stats_details AS (
    SELECT 
        'Bureaux de vote' as indicateur,
        CAST(total_bureaux_vote AS VARCHAR) as valeur,
        '' as unite
    FROM stats_globales
    
    UNION ALL
    
    SELECT 
        'Électeurs inscrits',
        CAST(total_inscrits AS VARCHAR),
        'personnes'
    FROM stats_globales
    
    UNION ALL
    
    SELECT 
        'Votants',
        CAST(total_votants AS VARCHAR),
        'personnes'
    FROM stats_globales
    
    UNION ALL
    
    SELECT 
        'Taux de participation',
        CAST(taux_participation_reel AS VARCHAR) || '%',
        ''
    FROM stats_globales
    
    UNION ALL
    
    SELECT 
        'Bulletins nuls',
        CAST(total_bulletins_nuls AS VARCHAR),
        'bulletins'
    FROM stats_globales
    
    UNION ALL
    
    SELECT 
        'Pourcentage de nuls',
        CAST(pourcentage_nuls AS VARCHAR) || '%',
        '(par rapport aux votants)'
    FROM stats_globales
    
    UNION ALL
    
    SELECT 
        'Suffrages exprimés',
        CAST(total_suffrages_exprimes AS VARCHAR),
        'voix'
    FROM stats_globales
    
    UNION ALL
    
    SELECT 
        'Bulletins blancs',
        CAST(total_bulletins_blancs AS VARCHAR),
        'bulletins'
    FROM stats_globales
    
    UNION ALL
    
    SELECT 
        'Pourcentage de blancs',
        CAST(pourcentage_blancs AS VARCHAR) || '%',
        '(par rapport aux votants)'
    FROM stats_globales
    
    UNION ALL
    
    SELECT 
        'Taux de validité',
        CAST(100 - pourcentage_nuls - pourcentage_blancs AS VARCHAR) || '%',
        '(bulletins valides)'
    FROM stats_globales
)
SELECT * FROM stats_details
ORDER BY 
    CASE indicateur
        WHEN 'Bureaux de vote' THEN 1
        WHEN 'Électeurs inscrits' THEN 2
        WHEN 'Votants' THEN 3
        WHEN 'Taux de participation' THEN 4
        WHEN 'Suffrages exprimés' THEN 5
        WHEN 'Bulletins nuls' THEN 6
        WHEN 'Pourcentage de nuls' THEN 7
        WHEN 'Bulletins blancs' THEN 8
        WHEN 'Pourcentage de blancs' THEN 9
        WHEN 'Taux de validité' THEN 10
        ELSE 11
    END
""").fetchdf()
print(q5_stats.to_string(index=False))

# Version formatée pour rapport
print(f"\n5. {questions['Q5']} - SYNTHÈSE FORMATÉE")
q5_summary = conn.execute("""
SELECT 
    CAST(SUM(total_bureaux_vote) AS BIGINT) as bureaux_vote,
    CAST(SUM(total_inscrits) AS BIGINT) as inscrits,
    CAST(SUM(total_votants) AS BIGINT) as votants,
    ROUND(SUM(total_votants) / SUM(total_inscrits) * 100, 2) as participation_pourcent,
    CAST(SUM(total_bulletins_nuls) AS BIGINT) as bulletins_nuls,
    (SELECT CAST(SUM(total_voix_exprimees) AS BIGINT) FROM vw_region_stats) as suffrages_exprimes,
    CAST(SUM(total_bulletins_blancs) AS BIGINT) as bulletins_blancs,
    ROUND(SUM(total_bulletins_nuls) / SUM(total_votants) * 100, 2) as pourcentage_nuls,
    ROUND(SUM(total_bulletins_blancs) / SUM(total_votants) * 100, 2) as pourcentage_blancs
FROM vw_region_stats
""").fetchdf()

summary = q5_summary.iloc[0]
print(f"""
SYNTHÈSE DES STATISTIQUES ÉLECTORALES :
----------------------------------------
• Bureaux de vote       : {summary['bureaux_vote']:,}
• Électeurs inscrits    : {summary['inscrits']:,}
• Votants               : {summary['votants']:,}
• Taux de participation : {summary['participation_pourcent']}%
• Bulletins nuls        : {summary['bulletins_nuls']:,} ({summary['pourcentage_nuls']}%)
• Suffrages exprimés    : {summary['suffrages_exprimes']:,}
• Bulletins blancs      : {summary['bulletins_blancs']:,} ({summary['pourcentage_blancs']}%)
• Taux de validité      : {100 - summary['pourcentage_nuls'] - summary['pourcentage_blancs']:.1f}%

RÉPARTITION DES VOIX :
• Voix valides : {summary['suffrages_exprimes']:,} (100%)
• Nuls         : {summary['bulletins_nuls']:,} ({summary['pourcentage_nuls']}%)
• Blancs       : {summary['bulletins_blancs']:,} ({summary['pourcentage_blancs']}%)
• Total bulletins : {summary['votants']:,}
""")

print(" TOUTES LES RÉPONSES DU CAHIER DES CHARGES SONT PRÊTES")

 QUESTIONS DU CAHIER DES CHARGES - RÉPONSES COMPLÈTES

1. Combien de sièges le RHDP a-t-il remportés ?
parti  sieges_remportes           resultat_formate  taux_victoire_pourcent                                                     interpretation
 RHDP             155.0 155 sièges sur 205 (75.6%)               75.609756 Le RHDP a remporté 155 sièges sur 205 circonscriptions nationales.

2. Top 10 des candidats par score dans la région AGNEBY-TIASSA
 rang                    candidat                   parti  voix  pourcentage_voix  statut
    1          DIMBA N'GOU PIERRE                    RHDP 10675             85.37     ÉLU
    2           KOFFI AKA CHARLES                    RHDP  9078             66.35     ÉLU
    3        SANOGO DRAMANE ALPHA                    RHDP  8557             52.42     ÉLU
    4      ASSALE TIEMOKO ANTOINE ADCI - GP-PAIX - VALEUR  7489             45.88 Non élu
    5      KOUADIO BEUGRE BERNARD                    RHDP  3912             54.51     ÉLU
    6    

In [ ]:
verif = conn.execute("""
SELECT 
    'Suffrages exprimés (somme voix)' as indicateur,
    SUM(total_voix_exprimees) as valeur
FROM vw_region_stats

UNION ALL

SELECT 
    'Bulletins blancs totaux',
    SUM(total_bulletins_blancs)
FROM vw_region_stats

UNION ALL

SELECT 
    'Bulletins nuls totaux',
    SUM(total_bulletins_nuls)
FROM vw_region_stats

UNION ALL

SELECT 
    'Votants totaux',
    SUM(total_votants)
FROM vw_region_stats

UNION ALL

SELECT 
    'Vérification: Votants = Blancs + Nuls + Exprimés',
    CASE 
        WHEN SUM(total_votants) = SUM(total_bulletins_blancs) + 
                                  SUM(total_bulletins_nuls) + 
                                  SUM(total_voix_exprimees)
        THEN ' OK'
        ELSE ' PROBLÈME'
    END
FROM vw_region_stats
""").fetchdf()

print(verif.to_string(index=False))

                                      indicateur    valeur
                 Suffrages exprimés (somme voix) 2913991.0
                         Bulletins blancs totaux   29578.0
                           Bulletins nuls totaux   68525.0
                                  Votants totaux 3012094.0
Vérification: Votants = Blancs + Nuls + Exprimés        OK


##### ***Génération automatique du schéma***

In [ ]:
import duckdb
import json
import pandas as pd
from datetime import datetime
from pathlib import Path

def generate_schema_and_save_data():
    """Génère le schéma JSON et sauvegarde les données intermédiaires"""
    
    # Chemins
    BASE_DIR = Path(r"E:\Projet_artefact")
    SRC_DIR = BASE_DIR / "src"
    DATA_DIR = SRC_DIR / "data"
    DB_DIR = DATA_DIR / "database"
    
    DB_PATH = DB_DIR / "election_ci.db"
    SCHEMA_PATH = Path(r"E:\Projet_artefact\src\schema_for_agent.json")
    
    # Créer les dossiers si nécessaire
    for directory in [SRC_DIR, DATA_DIR, DB_DIR]:
        directory.mkdir(parents=True, exist_ok=True)
    
    # Connexion à la base de données
    print(f" Connexion à : {DB_PATH}")
    conn = duckdb.connect(str(DB_PATH))
    
    # 1. STATISTIQUES GLOBALES
    print("\n Collecte des statistiques...")
    try:
        stats_query = """
        WITH stats AS (
            SELECT 
                COUNT(DISTINCT r.row_id) as total_candidatures,
                SUM(r.est_elu) as total_elus,
                COUNT(DISTINCT r.region) as total_regions,
                COUNT(DISTINCT r.circonscription_id) as total_circonscriptions,
                COUNT(DISTINCT r.parti_standardized) as total_partis,
                SUM(r.voix) as total_voix_exprimees
            FROM vw_results_clean r
        )
        SELECT * FROM stats
        """
        stats_df = conn.execute(stats_query).fetchdf()
        stats_dict = stats_df.iloc[0].to_dict() if not stats_df.empty else {}
    except Exception as e:
        print(f" Erreur statistiques: {e}")
        stats_dict = {}
    
    # 2. SAUVEGARDE DES DONNÉES INTERMÉDIAIRES
    print("\n Sauvegarde des données intermédiaires...")
    
    # Récupérer les données nettoyées
    try:
        df_clean = conn.execute("SELECT * FROM vw_results_clean").fetchdf()
    except Exception as e:
        print(f" Erreur récupération données: {e}")
        df_clean = pd.DataFrame()
    
    # Sauvegarde CSV
    clean_csv_path = DATA_DIR / "dataset_cleaned.csv"
    try:
        if not df_clean.empty:
            df_clean.to_csv(clean_csv_path, index=False, encoding='utf-8')
            print(f" CSV sauvegardé : {clean_csv_path}")
            print(f"   {len(df_clean):,} lignes, {len(df_clean.columns)} colonnes")
    except Exception as e:
        print(f" Erreur sauvegarde CSV: {e}")
    
    # 3. GÉNÉRATION DU SCHÉMA JSON
    print("\n Génération du schéma JSON...")
    
    # Structure du schéma
    schema_data = {
        "database_info": {
            "type": "DuckDB",
            "file": "election_ci.db",
            "path": str(DB_PATH.relative_to(BASE_DIR)),
            "created_at": datetime.now().isoformat(),
            "statistics": {k: float(v) for k, v in stats_dict.items()}
        },
        "allowed_views": [],
        "allowed_tables": [],
        "security_rules": {
            "allowed_operations": ["SELECT"],
            "forbidden_keywords": [
                'insert', 'update', 'delete', 'drop', 'alter', 'truncate',
                'create', 'grant', 'revoke', 'exec', 'execute', '--', '/*',
                'union select', 'information_schema', 'pg_', 'system',
                'merge', 'replace', 'commit', 'rollback'
            ],
            "auto_limit": 100,
            "timeout_ms": 5000,
            "max_rows": 10000
        },
        "example_queries": [
            "SELECT COUNT(*) as total_elus FROM vw_winners",
            "SELECT candidat, parti, voix FROM vw_results_clean ORDER BY voix DESC LIMIT 10",
            "SELECT region, ROUND(AVG(taux_de_participation), 2) as participation_moyenne FROM vw_turnout GROUP BY region ORDER BY participation_moyenne DESC",
            "SELECT parti, COUNT(*) as sieges FROM vw_winners GROUP BY parti ORDER BY sieges DESC",
            "SELECT candidat, parti, region, voix FROM vw_candidate_performance WHERE rang_circonscription = 1 ORDER BY voix DESC LIMIT 10",
            "SELECT region, total_votants, total_inscrits, taux_participation_moyen FROM vw_region_stats ORDER BY taux_participation_moyen DESC",
            "SELECT parti, total_candidats, total_voix, sieges_remportes FROM vw_party_stats ORDER BY sieges_remportes DESC",
            "SELECT parti, total_votes, seats_won, avg_vote_percentage FROM vw_party_search ORDER BY total_votes DESC LIMIT 5",
            "SELECT * FROM dim_party WHERE party_name LIKE '%RHDP%' OR known_aliases LIKE '%RHDP%'"
        ],
        "column_descriptions": {
            "vw_results_clean": {
                "region": "Nom de la région administrative",
                "circonscription": "Circonscription électorale (peut contenir plusieurs communes)",
                "parti": "Parti politique ou 'INDEPENDANT'",
                "parti_standardized": "Parti standardisé (regroupement des variantes)",
                "candidat": "Nom complet du candidat",
                "voix": "Nombre de voix obtenues (entier)",
                "pourcentage_voix": "Pourcentage de voix dans la circonscription",
                "elu": "Texte 'ELU(E)' si élu, vide sinon",
                "est_elu": "1 si élu, 0 sinon (pour calculs)",
                "inscrits": "Nombre d'électeurs inscrits (uniquement sur première ligne de chaque circonscription)",
                "votants": "Nombre de votants (uniquement sur première ligne de chaque circonscription)"
            },
            "vw_party_search": {
                "parti": "Nom standardisé du parti politique",
                "total_candidates": "Nombre total de candidats présentés par le parti",
                "seats_won": "Nombre de sièges remportés par le parti",
                "total_votes": "Nombre total de voix obtenues par le parti",
                "avg_vote_percentage": "Pourcentage moyen de voix par candidat",
                "known_aliases": "Alias connus du parti séparés par des virgules",
                "search_text": "Texte formaté pour la recherche sémantique (RAG)"
            },
            "dim_party": {
                "party_id": "Identifiant unique du parti",
                "party_name": "Nom standardisé du parti",
                "total_candidates": "Nombre total de candidats",
                "total_seats_won": "Nombre total de sièges remportés",
                "total_votes": "Nombre total de voix obtenues",
                "avg_vote_percentage": "Pourcentage moyen de voix par candidat"
            }
        },
        "common_aliases": {
            "partis": {
                "RHDP": ["R.H.D.P", "Rassemblement des Houphouëtistes", "RASSEMBLEMENT DES HOUPHOUËTISTES POUR LA DÉMOCRATIE ET LA PAIX"],
                "PDCI-RDA": ["PDCI", "RDA", "PARTI DÉMOCRATIQUE DE CÔTE D'IVOIRE"],
                "FPI": ["FRONT POPULAIRE IVOIRIEN"],
                "ADCI": ["ALLIANCE DES DÉMOCRATES DE CÔTE D'IVOIRE", "ADCI - GP-PAIX - VALEUR"],
                "MGC": ["MOUVEMENT DES GENS DE CÔTE D'IVOIRE"],
                "INDEPENDANT": ["INDÉPENDANT", "IND", "SANS ETIQUETTE", "CANDIDAT INDÉPENDANT"],
                "CNPCIN": [],
                "GJPA-CI": [],
                "CNJB-ADO": [],
                "PRO CI": ["PRO COTE DIVOIRE"],
                "CODE": [],
                "GP-PAIX": [],
                "AIDE": [],
                "PIA/PRI/CODE": [],
                "UNCI": [],
                "PDCI - FPI - ADCI": ["PDCI-FPI-ADCI"],
                "MERCI": [],
                "URCI": [],
                "LE BUFFLE": [],
                "UDCY": [],
                "EDS": [],
                "CRI PANAFRICAIN": [],
                "MDR": [],
                "ICON": [],
                "P.B.J.V": [],
                "UNPR": [],
                "REEL.CI": [],
                "APR.CI": [],
                "AIRD": [],
                "PPSD": [],
                "ACI": [],
                "MNRP": [],
                "FPP": [],
                "FNDR": [],
                "UFD": [],
                "MLPCI": [],
                "CNDCI": [],
                "UDP": [],
                "PIA/CODE": [],
                "PDCI-RDA - EDS": ["PDCI-RDA-EDS"],
                "MIRDEP": [],
                "RDP": []
            },
            "regions": {
                "AGNEBY-TIASSA": ["AGNEBY TIASSA", "AGNEBY"],
                "DISTRICT AUTONOME D'ABIDJAN": ["ABIDJAN", "DISTRICT ABIDJAN", "ABIDJAN DISTRICT"],
                "DISTRICT AUTONOME DE YAMOUSSOUKRO": ["YAMOUSSOUKRO", "DISTRICT YAMOUSSOUKRO", "YAMOUSSOUKRO DISTRICT"],
                "HAUT- SASSANDRA": ["HAUT SASSANDRA", "HAUT-SASSANDRA"],
                "INDENIE-DJUABLIN": ["INDENIE DJUABLIN", "INDENIE/DJUABLIN"],
                "LOH-DJIBOUA": ["LOH DJIBOUA", "LOH/DJIBOUA"],
                "SUD-COMOE": ["SUD COMOE", "SUD-COMÉ"]
            }
        },
        "column_aliases": {
            "vw_region_stats": {
                "taux_de_participation": ["taux_participation_moyen", "participation_moyenne", "participation"],
                "region": ["région", "nom_region"]
            },
            "vw_candidate_performance": {
                "parti": ["parti_standardized", "parti_politique", "groupe_politique"],
                "candidat": ["nom_candidat", "candidate"]
            },
            "vw_results_clean": {
                "parti_standardized": ["parti", "parti_politique", "groupe"]
            },
            "vw_party_search": {
                "parti": ["party_name", "parti_politique", "groupe_politique"],
                "seats_won": ["sieges_remportes", "nombre_sieges", "total_sieges"],
                "total_votes": ["voix_totales", "total_voix", "nombre_voix"]
            },
            "dim_party": {
                "party_name": ["parti", "nom_parti", "parti_standardized"],
                "total_seats_won": ["sieges_remportes", "elus", "total_elus"]
            }
        }
    }
    
    # 4. AJOUT DES VUES AU SCHÉMA
    print(" Récupération des informations des vues...")
    views_list = [
        "vw_results_clean", "vw_winners", "vw_turnout", 
        "vw_party_stats", "vw_region_stats", "vw_candidate_performance",
        "vw_rag_search", "vw_party_search"
    ]
    
    for view_name in views_list:
        try:
            check_view = conn.execute(f"""
            SELECT COUNT(*) as exists_flag 
            FROM information_schema.tables 
            WHERE table_name = '{view_name}' AND table_type = 'VIEW'
            """).fetchdf()
            
            if check_view.iloc[0]['exists_flag'] == 0:
                print(f"     {view_name} non trouvée")
                continue
                
            columns_info = conn.execute(f"""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = '{view_name}'
            ORDER BY ordinal_position
            """).fetchdf()
            
            if not columns_info.empty:
                view_info = {
                    "view_name": view_name,
                    "description": "",
                    "columns": []
                }
                
                descriptions = {
                    "vw_results_clean": "Vue principale avec toutes les données nettoyées et normalisées",
                    "vw_winners": "Vue contenant uniquement les candidats élus",
                    "vw_turnout": "Vue avec les statistiques de participation par circonscription (doublons éliminés)",
                    "vw_party_stats": "Vue avec les statistiques agrégées par parti politique",
                    "vw_region_stats": "Vue avec les statistiques agrégées par région",
                    "vw_candidate_performance": "Vue avec la performance détaillée et le classement des candidats",
                    "vw_rag_search": "Vue optimisée pour la recherche sémantique (RAG)",
                    "vw_party_search": "Vue pour la recherche de partis politiques avec aliases"
                }
                view_info["description"] = descriptions.get(view_name, "")
                
                for _, row in columns_info.iterrows():
                    view_info["columns"].append({
                        "name": row["column_name"],
                        "type": row["data_type"]
                    })
                
                schema_data["allowed_views"].append(view_info)
                print(f"    {view_name} ajoutée")
                
        except Exception as e:
            print(f"   {view_name}: {str(e)[:50]}")
    
    # 5. AJOUT DES TABLES
    print("\n Récupération des informations des tables...")
    tables_list = ["dim_party"]
    
    for table_name in tables_list:
        try:
            check_table = conn.execute(f"""
            SELECT COUNT(*) as exists_flag 
            FROM information_schema.tables 
            WHERE table_name = '{table_name}' AND table_type = 'BASE TABLE'
            """).fetchdf()
            
            if check_table.iloc[0]['exists_flag'] == 0:
                print(f"     Table {table_name} non trouvée")
                continue
                
            columns_info = conn.execute(f"""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = '{table_name}'
            ORDER BY ordinal_position
            """).fetchdf()
            
            if not columns_info.empty:
                table_info = {
                    "table_name": table_name,
                    "description": "Table dimension des partis politiques pour référence",
                    "columns": []
                }
                
                for _, row in columns_info.iterrows():
                    table_info["columns"].append({
                        "name": row["column_name"],
                        "type": row["data_type"]
                    })
                
                schema_data["allowed_tables"].append(table_info)
                print(f"   Table {table_name} ajoutée")
                
        except Exception as e:
            print(f"    Table {table_name}: {str(e)[:50]}")
    
    # 6. SAUVEGARDE DU SCHÉMA JSON
    print(f"\n Sauvegarde du schéma JSON...")
    try:
        with open(SCHEMA_PATH, 'w', encoding='utf-8') as f:
            json.dump(schema_data, f, indent=2, ensure_ascii=False)
        print(f" Schéma sauvegardé : {SCHEMA_PATH}")
        print(f" Taille : {SCHEMA_PATH.stat().st_size:,} octets")
    except Exception as e:
        print(f" Erreur sauvegarde JSON: {e}")
    
    # 7. SAUVEGARDE DU RÉSUMÉ STATISTIQUE
    print("\n Création du résumé statistique...")
    summary_path = DATA_DIR / "database_summary.txt"
    
    try:
        with open(summary_path, 'w', encoding='utf-8') as f:
            f.write(f"{'='*50}\n")
            f.write(f"BASE DE DONNÉES ÉLECTORALES - CÔTE D'IVOIRE\n")
            f.write(f"{'='*50}\n")
            f.write(f"Date de génération : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Base de données : {DB_PATH}\n")
            f.write(f"Schéma agent : {SCHEMA_PATH}\n\n")
            
            # Statistiques globales
            f.write(f"{'-'*40}\n")
            f.write("STATISTIQUES GLOBALES\n")
            f.write(f"{'-'*40}\n")
            if stats_dict:
                for key, value in stats_dict.items():
                    f.write(f"• {key.replace('_', ' ').title()}: {value:,}\n")
            
            f.write(f"\n{'-'*40}\n")
            f.write("VUES DISPONIBLES\n")
            f.write(f"{'-'*40}\n")
            for view in views_list:
                try:
                    count = conn.execute(f"SELECT COUNT(*) FROM {view}").fetchone()[0]
                    f.write(f"• {view}: {count:,} lignes\n")
                except:
                    f.write(f"• {view}: NON DISPONIBLE\n")
            
            f.write(f"\n{'-'*40}\n")
            f.write("TOP 5 DES PARTIS PAR SIÈGES\n")
            f.write(f"{'-'*40}\n")
            try:
                parties = conn.execute("""
                SELECT parti, sieges_remportes, total_voix
                FROM vw_party_stats 
                ORDER BY sieges_remportes DESC 
                LIMIT 5
                """).fetchdf()
                for _, row in parties.iterrows():
                    f.write(f"• {row['parti']}: {row['sieges_remportes']} sièges, {row['total_voix']:,} voix\n")
            except:
                f.write("Données non disponibles\n")
            
            f.write(f"\n{'-'*40}\n")
            f.write("FICHIERS GÉNÉRÉS\n")
            f.write(f"{'-'*40}\n")
            f.write(f"• Base de données : {DB_PATH}\n")
            f.write(f"• Données nettoyées : {clean_csv_path}\n")
            f.write(f"• Schéma agent : {SCHEMA_PATH}\n")
            f.write(f"• Résumé : {summary_path}\n")
            
            f.write(f"\n{'='*50}\n")
            f.write("GÉNÉRATION TERMINÉE AVEC SUCCÈS\n")
            f.write(f"{'='*50}\n")
        
        print(f" Résumé sauvegardé : {summary_path}")
    except Exception as e:
        print(f" Erreur création résumé: {e}")
    
    # 8. VÉRIFICATION FINALE
    print(" VÉRIFICATION FINALE")
    
    # Test de cohérence
    try:
        coherency = conn.execute("""
        SELECT 
            COUNT(DISTINCT circonscription_id) as circonscriptions,
            SUM(est_elu) as elus,
            CASE 
                WHEN COUNT(DISTINCT circonscription_id) = SUM(est_elu) 
                THEN ' CORRECT' 
                ELSE ' PROBLÈME' 
            END as statut
        FROM vw_results_clean
        """).fetchdf()
        print("1. Cohérence élus/circonscriptions :")
        print(coherency.to_string(index=False))
    except Exception as e:
        print(f" Erreur cohérence: {e}")
    
    # Test des vues principales
    print("\n2. Tests des vues principales :")
    test_views = ["vw_winners", "vw_turnout", "vw_party_stats"]
    for view in test_views:
        try:
            count = conn.execute(f"SELECT COUNT(*) FROM {view}").fetchone()[0]
            print(f"    {view}: {count:,} lignes")
        except Exception as e:
            print(f"    {view}: {str(e)[:50]}")
    
    # Statistiques globales finales
    print("\n3. Résumé final :")
    try:
        final_stats = conn.execute("""
        SELECT 
            'Circonscriptions' as indicateur,
            COUNT(DISTINCT circonscription_id) as valeur
        FROM vw_results_clean
        UNION ALL
        SELECT 'Élus', SUM(est_elu) FROM vw_results_clean
        UNION ALL
        SELECT 'Partis', COUNT(DISTINCT parti_standardized) FROM vw_results_clean
        UNION ALL
        SELECT 'Candidats', COUNT(*) FROM vw_results_clean
        """).fetchdf()
        for _, row in final_stats.iterrows():
            print(f"   • {row['indicateur']}: {row['valeur']:,}")
    except Exception as e:
        print(f" Erreur statistiques finales: {e}")
    
    # 9. FERMETURE ET RÉSUMÉ
    conn.close()
    
    print(" PROCESSUS TERMINÉ AVEC SUCCÈS")
    print(f" Base de données : {DB_PATH}")
    print(f" Schéma agent : {SCHEMA_PATH}")
    print(f" Données nettoyées : {clean_csv_path}")
    print(f" Résumé statistique : {summary_path}")
    
    if not df_clean.empty:
        print(f"\n DONNÉES NETTOYÉES :")
        print(f"   • Lignes : {len(df_clean):,}")
        print(f"   • Colonnes : {len(df_clean.columns)}")
    
    print(f"\n SCHÉMA GÉNÉRÉ :")
    print(f"   • Vues documentées : {len(schema_data['allowed_views'])}")
    print(f"   • Tables documentées : {len(schema_data['allowed_tables'])}")
    print(f"   • Requêtes d'exemple : {len(schema_data['example_queries'])}")
    
    return {
        "schema": schema_data,
        "clean_data": df_clean,
        "stats": stats_dict,
        "files": {
            "database": DB_PATH,
            "schema": SCHEMA_PATH,
            "clean_csv": clean_csv_path,
            "summary": summary_path
        }
    }



In [ ]:
print(" GÉNÉRATION DU SCHÉMA ET SAUVEGARDE DES DONNÉES")

try:
    result = generate_schema_and_save_data()
    
    print("\n FICHIERS CRÉÉS :")
    for name, path in result["files"].items():
        if path.exists():
            print(f"   ✓ {name}: {path}")
    
    print("\n Le système est maintenant prêt pour :")
    print("   1. L'agent AI (utilise le schéma JSON)")
    print("   2. Les analyses (utilise les données CSV)")
    print("   3. Les requêtes SQL (utilise la base DuckDB)")
    
except Exception as e:
    print(f"\n ERREUR CRITIQUE: {e}")
    print("\n Vérifiez que :")
    print("   • La base de données existe")
    print("   • Les vues ont été créées")
    print("   • Vous avez les permissions d'écriture")


 GÉNÉRATION DU SCHÉMA ET SAUVEGARDE DES DONNÉES
 Connexion à : E:\Projet_artefact\src\data\database\election_ci.db

 Collecte des statistiques...

 Sauvegarde des données intermédiaires...
 CSV sauvegardé : E:\Projet_artefact\src\data\dataset_cleaned.csv
   1,125 lignes, 26 colonnes

 Génération du schéma JSON...
 Récupération des informations des vues...
    vw_results_clean ajoutée
    vw_winners ajoutée
    vw_turnout ajoutée
    vw_party_stats ajoutée
    vw_region_stats ajoutée
    vw_candidate_performance ajoutée
    vw_rag_search ajoutée
    vw_party_search ajoutée

 Récupération des informations des tables...
   Table dim_party ajoutée

 Sauvegarde du schéma JSON...
 Schéma sauvegardé : E:\Projet_artefact\src\schema_for_agent.json
 Taille : 18,120 octets

 Création du résumé statistique...
 Résumé sauvegardé : E:\Projet_artefact\src\data\database_summary.txt
 VÉRIFICATION FINALE
1. Cohérence élus/circonscriptions :
 circonscriptions  elus   statut
              205 205.0  CORRE

In [27]:
conn.close()
print(" Connexion fermée")

 Connexion fermée
